This notebook shows two simple examples of Collaborative Filtering using user-based and item-based approaches. The source data is from [this]('http://files.grouplens.org/datasets/movielens/ml-latest-small.zip') link.

### Import packages

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import pairwise_distances 
import random 

### Set-up

In [2]:
# Location and filenames of the input data

ratings_file = 'https://www.dropbox.com/s/g294u2bqsuny36p/ratings.csv?dl=1'

movies_file = 'https://www.dropbox.com/s/yk30jezunlsb72i/movies.csv?dl=1'

random.seed(5)

### Read data

In [3]:
# Ratings data set

ratings_df = pd.read_csv(ratings_file)

ratings_df.shape

(100836, 4)

In [4]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
# Movies data set

movies_df = pd.read_csv(movies_file)

movies_df.shape

(9742, 3)

In [6]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### Prepare data

Let's drop columns that we don't need, and then combine those two datasets.

In [7]:
ratings_df = ratings_df.drop('timestamp', axis=1)

movies_df = movies_df.drop('genres', axis=1)

df = ratings_df.merge(movies_df, on='movieId', how='inner')

df.shape

(100836, 4)

In [8]:
df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [9]:
df.dtypes

userId       int64
movieId      int64
rating     float64
title       object
dtype: object

### Matrix representation

Collaborative Filtering requires the data to be in a user-item matrix format.

In [10]:
user_item_df = df.pivot_table(index='userId', columns='title', values='rating')

user_item_df.shape

(610, 9719)

In [11]:
user_item_df.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


For the purpose of this analysis, we will restrict the number of movies to 500.

In [11]:
# Take the most rated top 500 movies

top_500_movies = df['title'].value_counts()[:500].index

In [13]:
top_500_movies

Index(['Forrest Gump (1994)', 'Shawshank Redemption, The (1994)',
       'Pulp Fiction (1994)', 'Silence of the Lambs, The (1991)',
       'Matrix, The (1999)', 'Star Wars: Episode IV - A New Hope (1977)',
       'Jurassic Park (1993)', 'Braveheart (1995)',
       'Terminator 2: Judgment Day (1991)', 'Schindler's List (1993)',
       ...
       'Basic Instinct (1992)', 'Rumble in the Bronx (Hont faan kui) (1995)',
       'Lethal Weapon 3 (1992)', 'Blue Velvet (1986)', 'Gran Torino (2008)',
       'Beverly Hills Cop (1984)', 'Mortal Kombat (1995)',
       'Fear and Loathing in Las Vegas (1998)', 'The Butterfly Effect (2004)',
       'Pianist, The (2002)'],
      dtype='object', length=500)

In [12]:
# Subset the uset_item_df dataframe by taking only those top 500 titles (columns)

user_item_df = user_item_df[top_500_movies]

user_item_df.head()

title,Forrest Gump (1994),"Shawshank Redemption, The (1994)",Pulp Fiction (1994),"Silence of the Lambs, The (1991)","Matrix, The (1999)",Star Wars: Episode IV - A New Hope (1977),Jurassic Park (1993),Braveheart (1995),Terminator 2: Judgment Day (1991),Schindler's List (1993),...,Basic Instinct (1992),Rumble in the Bronx (Hont faan kui) (1995),Lethal Weapon 3 (1992),Blue Velvet (1986),Gran Torino (2008),Beverly Hills Cop (1984),Mortal Kombat (1995),Fear and Loathing in Las Vegas (1998),The Butterfly Effect (2004),"Pianist, The (2002)"
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,3.0,4.0,5.0,5.0,4.0,4.0,NaN,5.0,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,1.0,5.0,1.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,5.0,NaN,NaN,NaN,NaN,4.0,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Reset index to remove the multi-level index

user_item_df = user_item_df.reset_index()

user_item_df.head()

title,userId,Forrest Gump (1994),"Shawshank Redemption, The (1994)",Pulp Fiction (1994),"Silence of the Lambs, The (1991)","Matrix, The (1999)",Star Wars: Episode IV - A New Hope (1977),Jurassic Park (1993),Braveheart (1995),Terminator 2: Judgment Day (1991),...,Basic Instinct (1992),Rumble in the Bronx (Hont faan kui) (1995),Lethal Weapon 3 (1992),Blue Velvet (1986),Gran Torino (2008),Beverly Hills Cop (1984),Mortal Kombat (1995),Fear and Loathing in Las Vegas (1998),The Butterfly Effect (2004),"Pianist, The (2002)"
0,1,4.0,NaN,3.0,4.0,5.0,5.0,4.0,4.0,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,1.0,5.0,1.0,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,3.0,5.0,NaN,NaN,NaN,NaN,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's take a quick look at the top rated movies.

In [16]:
avg_ratings = user_item_df.mean().sort_values(ascending=False)

avg_ratings.head(20)

title
userId                                                                         305.500000
Shawshank Redemption, The (1994)                                                 4.429022
Godfather, The (1972)                                                            4.289062
Fight Club (1999)                                                                4.272936
Cool Hand Luke (1967)                                                            4.271930
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)      4.268041
Rear Window (1954)                                                               4.261905
Godfather: Part II, The (1974)                                                   4.259690
Departed, The (2006)                                                             4.252336
Goodfellas (1990)                                                                4.250000
Casablanca (1942)                                                                4.240000
Dark

Let's check the ratings of a specific movie.

In [17]:
avg_ratings[avg_ratings.index.str.contains('Groundhog')]

title
Groundhog Day (1993)    3.944056
dtype: float64

What % of this entire population saw this movie?

In [18]:
user_item_df['Groundhog Day (1993)'].notnull().sum() / len(user_item_df)

0.23442622950819672

How many users have *not* watched this movie?

In [19]:
user_item_df['Groundhog Day (1993)'].isnull().sum()

467

### User-based Collaborative Filtering

Let's randomly choose one user who has not watched *Groundhog Day.* We will then proceed to predict his/her rating for this movie.

In [20]:
target_movie = 'Groundhog Day (1993)'

# First take all users who have not watched this movie

target_users = user_item_df[user_item_df[target_movie].isnull()]

target_users.shape

(467, 501)

In [22]:
target_users

title,userId,Forrest Gump (1994),"Shawshank Redemption, The (1994)",Pulp Fiction (1994),"Silence of the Lambs, The (1991)","Matrix, The (1999)",Star Wars: Episode IV - A New Hope (1977),Jurassic Park (1993),Braveheart (1995),Terminator 2: Judgment Day (1991),...,Basic Instinct (1992),Rumble in the Bronx (Hont faan kui) (1995),Lethal Weapon 3 (1992),Blue Velvet (1986),Gran Torino (2008),Beverly Hills Cop (1984),Mortal Kombat (1995),Fear and Loathing in Las Vegas (1998),The Butterfly Effect (2004),"Pianist, The (2002)"
1,2,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,3.0,5.0,NaN,NaN,NaN,NaN,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,5.0,5.0,2.0,4.0,NaN,NaN,5.0,5.0,3.0,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,5.0,NaN,NaN,5.0,NaN,5.0,5.0,NaN,2.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,3.0,5.0,4.0,4.0,NaN,NaN,4.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,3.5,NaN,1.0,NaN,0.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,11,5.0,4.0,NaN,5.0,NaN,NaN,4.0,5.0,4.0,...,NaN,NaN,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
12,13,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Randomly select one user from this group

target_user = random.sample(list(target_users.index), 1)

target_user

[418]

This is our **target** user, for whom we wish to predict the rating for *Groundhog Day*.

Let's calculate the distance from this user to all other users.

In [26]:
movies_rated_by_target_user = user_item_df[user_item_df['userId'] == target_user].dropna(axis=1)

movies_rated_by_target_user.T

,417
title,
userId,418.0
Forrest Gump (1994),4.0
"Shawshank Redemption, The (1994)",5.0
Pulp Fiction (1994),4.5
"Silence of the Lambs, The (1991)",4.0
Schindler's List (1993),4.5
Fight Club (1999),5.0
"Usual Suspects, The (1995)",4.5
"Lord of the Rings: The Fellowship of the Ring, The (2001)",5.0


In [27]:
user_item_df.head()

title,userId,Forrest Gump (1994),"Shawshank Redemption, The (1994)",Pulp Fiction (1994),"Silence of the Lambs, The (1991)","Matrix, The (1999)",Star Wars: Episode IV - A New Hope (1977),Jurassic Park (1993),Braveheart (1995),Terminator 2: Judgment Day (1991),...,Basic Instinct (1992),Rumble in the Bronx (Hont faan kui) (1995),Lethal Weapon 3 (1992),Blue Velvet (1986),Gran Torino (2008),Beverly Hills Cop (1984),Mortal Kombat (1995),Fear and Loathing in Las Vegas (1998),The Butterfly Effect (2004),"Pianist, The (2002)"
0,1,4.0,NaN,3.0,4.0,5.0,5.0,4.0,4.0,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,1.0,5.0,1.0,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,3.0,5.0,NaN,NaN,NaN,NaN,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# All movies rated by the target user 
rated_movies_by_target_user = movies_rated_by_target_user.columns

# Create a mask to select all users but the target user, and users who have rated the target movie
mask = (user_item_df['userId'] != target_user) & (user_item_df[target_movie].notnull())

# Apply the filter -- take all users ID's that satisfy this criteria
user_search_space = user_item_df[mask]['userId'].values

# Take all users from this search space, and get their ratings for all movies that our target user rated
X = user_item_df[mask][rated_movies_by_target_user].fillna(0)

# Take the target use, and get their ratings for all movies sans the target movie
y = movies_rated_by_target_user

len(X), len(y)

(143, 1)

In [30]:
y.head()

title,userId,Forrest Gump (1994),"Shawshank Redemption, The (1994)",Pulp Fiction (1994),"Silence of the Lambs, The (1991)",Schindler's List (1993),Fight Club (1999),"Usual Suspects, The (1995)","Lord of the Rings: The Fellowship of the Ring, The (2001)","Lord of the Rings: The Two Towers, The (2002)",...,"Goonies, The (1985)","Social Network, The (2010)",Cool Hand Luke (1967),The Hunger Games (2012),Kung Fu Panda (2008),Mulholland Drive (2001),The Imitation Game (2014),Harry Potter and the Deathly Hallows: Part 2 (2011),Sixteen Candles (1984),The Butterfly Effect (2004)
417,418,4.0,5.0,4.5,4.0,4.5,5.0,4.5,5.0,5.0,...,4.0,4.5,5.0,4.0,0.5,3.0,4.5,4.5,3.5,4.5


In [32]:
user_dist = pairwise_distances(X, y, metric='cosine')

user_dist[:10]

array([[0.9127351 ],
       [0.67166026],
       [0.07655944],
       [0.28180598],
       [0.1902599 ],
       [0.24310966],
       [0.13248563],
       [0.03878952],
       [0.07399488],
       [0.07377764]])

Find the use that is "closest" to the target user.

In [33]:
min_dist = [user_dist[user_dist == user_dist.min()]][0][0]

min_dist

0.0005249721548548081

In [34]:
import numpy as np

closest_user = user_search_space[np.argmin(user_dist)]

closest_user

414

In [43]:
user_item_df[user_item_df['userId'] == closest_user].T[:5]

,413
title,
userId,414.0
Forrest Gump (1994),5.0
"Shawshank Redemption, The (1994)",5.0
Pulp Fiction (1994),5.0
"Silence of the Lambs, The (1991)",4.0


In [38]:
user_item_df[user_item_df['userId'] == target_user].T[:7]

,417
title,
userId,418.0
Forrest Gump (1994),4.0
"Shawshank Redemption, The (1994)",5.0
Pulp Fiction (1994),4.5
"Silence of the Lambs, The (1991)",4.0
"Matrix, The (1999)",NaN
Star Wars: Episode IV - A New Hope (1977),NaN


In [44]:
predicted_rating = user_item_df[user_item_df['userId'] == closest_user][target_movie]

predicted_rating

413    4.0
Name: Groundhog Day (1993), dtype: float64

This is a simple demonstartion of how user-user distance can be used to predict ratings (and recommend movies). This approach can be extended further to improve results.

### Item-based Collaborative Filtering

In [45]:
# List of all movies other than the target movie
all_other_movies = [col for col in user_item_df.columns if col not in (target_movie, 'userId')]

X = user_item_df[all_other_movies].fillna(0)

y = user_item_df[target_movie].fillna(0)

In [52]:
all_other_movies

['Forrest Gump (1994)',
 'Shawshank Redemption, The (1994)',
 'Pulp Fiction (1994)',
 'Silence of the Lambs, The (1991)',
 'Matrix, The (1999)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Jurassic Park (1993)',
 'Braveheart (1995)',
 'Terminator 2: Judgment Day (1991)',
 "Schindler's List (1993)",
 'Fight Club (1999)',
 'Toy Story (1995)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Usual Suspects, The (1995)',
 'American Beauty (1999)',
 'Seven (a.k.a. Se7en) (1995)',
 'Independence Day (a.k.a. ID4) (1996)',
 'Apollo 13 (1995)',
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
 'Lord of the Rings: The Fellowship of the Ring, The (2001)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Godfather, The (1972)',
 'Fugitive, The (1993)',
 'Batman (1989)',
 'Lord of the Rings: The Two Towers, The (2002)',
 'Saving Private Ryan (1998)',
 'Lord of the Rings: The Return of the King, The (2003)',
 'Aladdin (1992)',
 'Fargo (1996)',


In [46]:
item_dist = pairwise_distances(X.T, y.values.reshape(-1,610), metric='cosine')

item_dist[:10]

array([[0.4774823 ],
       [0.50736902],
       [0.52567167],
       [0.53229855],
       [0.46384951],
       [0.43052535],
       [0.52281557],
       [0.59504138],
       [0.54181274],
       [0.55983412]])

In [53]:
min_item_dist = [item_dist[item_dist == item_dist.min()]][0][0]

min_item_dist

0.38764197403915945

In [54]:
# List of all movies sorted by distance (shortest first)

[title for d, title in sorted(zip(item_dist, all_other_movies))][:10]

['Back to the Future (1985)',
 'Monty Python and the Holy Grail (1975)',
 'Men in Black (a.k.a. MIB) (1997)',
 'Princess Bride, The (1987)',
 "Ferris Bueller's Day Off (1986)",
 'Being John Malkovich (1999)',
 'Ghostbusters (a.k.a. Ghost Busters) (1984)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Indiana Jones and the Last Crusade (1989)',
 'Star Wars: Episode IV - A New Hope (1977)']

In [55]:
closest_item = all_other_movies[np.argmin(item_dist)]

closest_item

'Back to the Future (1985)'

Calculate the averate rating for this title. This will become our predicted rating for *Groundhog Day*.

In [56]:
predicted_rating_2 = user_item_df[closest_item].mean()

round(predicted_rating_2, 1)

4.0

This is a simple demonstartion of how item-item distance can be used to predict ratings (and recommend movies). This approach can be extended further to improve results.